In [1]:
from refactor.probes import model_setup
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-4fb110c0f6314175.parquet', 'val': 'data/val-00000-of-00001-da38a96348bd1590.parquet', 'test': 'data/test-00000-of-00001-fe5a9a4940bb3673.parquet'}
df = pd.read_parquet("hf://datasets/alexandrainst/nordjylland-news-summarization/" + splits["train"])

In [16]:
prompt = "Følgende er dokumenter med tilhørende resuméer."

for text, summ in zip(df.text.iloc[0:3], df.summary.iloc[0:3]):
    prompt+= f" Dokument: {text}\nResumé: {summ}"
doc = df.text.iloc[5]
prompt += f"Dokument: {doc}\n\nResumé:"

In [3]:
model_name = "AI-Sweden-Models/gpt-sw3-356m"
model, tokenizer, device = model_setup(model_name)


found device: cpu


In [14]:
prompt[-100:]

'lsen og drøfter mulige løsninger på det, siger Ole Ørnbøl.\n\nSkriv et resumé af ovenstående dokument.'

In [18]:
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)
generated_token_ids = model.generate(inputs=input_ids, max_new_tokens=120, do_sample=True)[0]
summary = tokenizer.decode(generated_token_ids)


In [19]:
summary[len(prompt)-10:]

'esumé: - De har fået lavet en lokalplan med en stor mængde boliger. Det vil på sigt medføre mere trafik og flere parkeringspladser, siger Jette Thøgersen, som er en af dem, der har skrevet under på lokalplanen for Bilka-centret. Nu håber hun dog på, at lokalplanen for Bilka-centret kan blive ved magten i Hjørring Byråd, selv om de har lavet indsigelse. Jette Thøgersen. Foto: TV2 Nord. Lokaler, som det er aftalt en dagligvarebutik at få 50 parkeringspladser til. Det vil få flere bil'

In [21]:
print(doc)

Balladen om et Bilka i Hjørring i stedet for A-Z har raset i årevis. Hjørring Kommune har lavet en lokalplan, som vil placeret et bydelscenter med salg af fødevarer på Ringvejen i Hjørring. Den lokalplan har Naturstyrelsen nu gjort indsigelser imod. En rundspørge, som TV2/Nord har lavet, viser også, at en stribe handelsstandsforeninger i de større og mindre byer rundt om Hjørring vil gøre indsigelser imod Bilka-projektet. De frygter nemlig butiksdød. Medlem af SF fra Hjørring Byråd, Jørgen Bing, græder ikke over Naturstyrelsens indsigelse, der er sendt til Hjørring Kommune den 28. marts. - Vi er meget tilfredse med Naturstyrelsens indsigelse i SF. Vi mener stadig, at et Bilkacenter i Hjørring vil skade mange bymidter i området, siger Jørgen Bing til TV2/Nord. Ole Ørnbøl, der er formand for Teknik og Miljøudvalget i Hjørring Kommune ser ikke indsigelsen som et endegyldigt stop for planerne om et Bilka. - Det er ikke usædvanligt, at Naturstyrelsen kommer med en veto-indsigelse. Teknisk f

In [8]:
len(summary.split(" "))

1185

In [6]:
print(summary)

 Følgende er dokumenter med tilhørende resuméer. Dokument: Opdatering: Manden er nu fundet af Nordjyllands Politi Den 8. oktober kunne politiet oplyse i en pressemeddelelse, at mandenvar meldt savnet, og nye oplysninger i sagenfår nu endnu en gang politiet til at henvende sig til offentligheden. - Det viser sig nemlig, at eftersøgte tidligere har boet i en årrække i det område, hvor hans bil, en metalblå Skoda Fabia, blev fundet, fortæller leder af politiets efterforskning, politikommissær Mikkel Mitchell. Bilen blev fundet parkeret på Gammel Strandvej i Aalborg natten til fredag den9. oktober 2020, og eftersøgte har boet i området fra 2000-2013 – nærmere bestemt på Brohusgade, oplyser Politiet i en pressemeddelelse. - Derfor kan vi ikke udelukke, at han har taget ophold i det område. Og vi opfordrer beboere i området omkring Brohusgade til at kigge en ekstra gang i deres skure, kælderum og lignende, siger politikommissær Mikkel Mitchell, der oplyser, at politiets efterforskning i sage

In [22]:
from evaluate import load
bertscore = load("bertscore")
predictions = [summary]
references = [doc]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print(results)


Using the latest cached version of the module from /Users/marki/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bertscore/cf4907b18f8f741f202232c0f8009a3bd49ff98802c245abcb6ea51a37a8c05b (last modified on Wed Nov 13 12:39:33 2024) since it couldn't be found locally at evaluate-metric--bertscore, or remotely on the Hugging Face Hub.


{'precision': [0.8027646541595459], 'recall': [0.8139000535011292], 'f1': [0.8082940578460693], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.51.3)'}
